In [1]:
import os
import pandas as pd
import json

from ast import literal_eval
from itertools import product
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from ast import literal_eval

In [4]:
test_file = "../datasets/inversion_language_confusion/langdist_data_all_langs/dataset2langdist_line_level_multi_0.3.csv"

In [5]:
df = pd.read_csv(test_file)

In [6]:
df

,model,training,eval_lang,step,pred_langs,emb_cos_sim
0,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",deu_Latn,Labels,{'deu': 1.0},NaN
1,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",deu_Latn,Base,"{'deu': 0.98, 'eng': 0.02}",0.917307
2,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",deu_Latn,Step1,"{'deu': 0.97, 'eng': 0.01}",0.919171
3,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",deu_Latn,Step50+sbeam8,"{'deu': 0.96, 'eng': 0.01}",0.947147
4,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",ydd_Hebr,Labels,{'heb': 1.0},NaN
...,...,...,...,...,...,...
509,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",mhr_Cyrl,Step50+sbeam8,"{'kaz': 0.73, 'rus': 0.21, 'mon': 0.02, 'mkd':...",0.890884
510,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",fin_Latn,Labels,{'fin': 0.99},NaN
511,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",fin_Latn,Base,"{'fin': 0.36, 'tur': 0.51, 'eng': 0.02, 'kaz':...",0.909733
512,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",fin_Latn,Step1,"{'tur': 0.79, 'fin': 0.14, 'aze': 0.01, 'eng':...",0.897174


In [7]:
df = df[df["step"]!="Labels"]

In [8]:
df["pred_langs"]= df["pred_langs"].apply(literal_eval)

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_47713/289928680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pred_langs"]= df["pred_langs"].apply(literal_eval)


In [9]:
df

,model,training,eval_lang,step,pred_langs,emb_cos_sim
1,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",deu_Latn,Base,"{'deu': 0.98, 'eng': 0.02}",0.917307
2,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",deu_Latn,Step1,"{'deu': 0.97, 'eng': 0.01}",0.919171
3,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",deu_Latn,Step50+sbeam8,"{'deu': 0.96, 'eng': 0.01}",0.947147
5,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",ydd_Hebr,Base,"{'mon': 0.1, 'kaz': 0.85, 'eng': 0.01, 'ukr': ...",0.893727
6,me5_cyrl-script,"['kaz_Cyrl', 'mon_Cyrl']",ydd_Hebr,Step1,"{'mon': 0.3, 'kaz': 0.64, 'eng': 0.01, 'bel': ...",0.907558
...,...,...,...,...,...,...
508,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",mhr_Cyrl,Step1,"{'kaz': 0.74, 'rus': 0.21, 'bel': 0.01, 'mkd':...",0.883512
509,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",mhr_Cyrl,Step50+sbeam8,"{'kaz': 0.73, 'rus': 0.21, 'mon': 0.02, 'mkd':...",0.890884
511,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",fin_Latn,Base,"{'fin': 0.36, 'tur': 0.51, 'eng': 0.02, 'kaz':...",0.909733
512,me5_turkic-fami,"['tur_Latn', 'kaz_Cyrl']",fin_Latn,Step1,"{'tur': 0.79, 'fin': 0.14, 'aze': 0.01, 'eng':...",0.897174


In [11]:
probs_df = pd.DataFrame(df['pred_langs'].tolist(), index=df.index).fillna(0)

In [12]:
len(df), len(probs_df)

(374, 374)

In [72]:
df_ = pd.concat([df, probs_df], axis=1)

In [73]:
df_["training"]= df_["training"].apply(literal_eval)

In [74]:
df_["pred_langs"].tolist()[0]["deu_Latn"]

0.87

In [75]:
df_.head(2)

,model,training,eval_lang,step,pred_langs,emb_cos_sim,deu_Latn,jpn_Jpan,cmn_Hani,eng_Latn,...,arb_Arab,hun_Latn,fin_Latn,pan_Guru,guj_Gujr,heb_Hebr,kaz_Cyrl,mon_Cyrl,hin_Deva,kor_Hang
1,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",deu_Latn,Step1,"{'deu_Latn': 0.87, 'jpn_Jpan': 0.07}",0.929772,0.87,0.07,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",deu_Latn,Step50+sbeam8,"{'deu_Latn': 0.82, 'cmn_Hani': 0.07}",0.913244,0.82,0.00,0.07,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
langs= probs_df.columns.tolist()

In [77]:
len(langs)

16

In [78]:
training_list=[]
eval_lang_list=[]
pred_others=[]

for train_langs, eval_lang, pred_langs in zip(df_["training"], df_["eval_lang"], df_["pred_langs"]):
        train_langs_eval = train_langs+[eval_lang]
        langs_related_to_training = set(train_langs_eval)
        pred_langs_set= set(list(pred_langs.keys()))
        pred_other = pred_langs_set -langs_related_to_training
    
        if len(pred_other) and 'eng_Latn' not in pred_other:
            print(f"training: {train_langs}, eval: {eval_lang}, pred: {pred_langs_set}, extra pred:{pred_other}")
        # else:
        #     print(train_langs_eval, pred_langs_list, pred_other)
    # except Exception as msg:
    #     print(train_langs, eval_lang)
    


training: ['tur_Latn', 'urd_Arab'], eval: heb_Hebr, pred: {'arb_Arab', 'tur_Latn', 'urd_Arab'}, extra pred:{'arb_Arab'}
training: ['tur_Latn', 'urd_Arab'], eval: heb_Hebr, pred: {'arb_Arab', 'tur_Latn', 'urd_Arab'}, extra pred:{'arb_Arab'}
training: ['heb_Hebr', 'arb_Arab'], eval: mhr_Cyrl, pred: {'arb_Arab', 'mon_Cyrl', 'kaz_Cyrl'}, extra pred:{'mon_Cyrl', 'kaz_Cyrl'}
training: ['heb_Hebr', 'arb_Arab'], eval: mhr_Cyrl, pred: {'mon_Cyrl', 'kaz_Cyrl'}, extra pred:{'mon_Cyrl', 'kaz_Cyrl'}
training: ['heb_Hebr', 'arb_Arab'], eval: mhr_Cyrl, pred: {'arb_Arab', 'mon_Cyrl', 'kaz_Cyrl'}, extra pred:{'mon_Cyrl', 'kaz_Cyrl'}
training: ['heb_Hebr', 'arb_Arab'], eval: fin_Latn, pred: {'deu_Latn', 'fin_Latn'}, extra pred:{'deu_Latn'}
training: ['heb_Hebr', 'arb_Arab'], eval: fin_Latn, pred: {'heb_Hebr', 'deu_Latn', 'fin_Latn'}, extra pred:{'deu_Latn'}
training: ['heb_Hebr', 'arb_Arab'], eval: fin_Latn, pred: {'arb_Arab', 'deu_Latn', 'heb_Hebr', 'fin_Latn'}, extra pred:{'deu_Latn'}
training: ['tur_

In [27]:
df_

,model,training,eval_lang,step,pred_langs,emb_cos_sim,deu_Latn,eng_Latn,cmn_Hani,jpn_Jpan,...,fin_Latn,urd_Arab,arb_Arab,kaz_Cyrl,pan_Guru,guj_Gujr,hin_Deva,heb_Hebr,mon_Cyrl,kor_Hang
1,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",deu_Latn,Step1,"{'deu_Latn': 0.67, 'eng_Latn': 0.06}",0.929772,0.67,0.06,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",deu_Latn,Step50+sbeam8,"{'deu_Latn': 0.69, 'eng_Latn': 0.07}",0.913244,0.69,0.07,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",deu_Latn,Base,{'deu_Latn': 0.64},0.910734,0.64,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",ydd_Hebr,Step1,"{'cmn_Hani': 0.66, 'eng_Latn': 0.23}",0.859916,0.00,0.23,0.66,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",ydd_Hebr,Step50+sbeam8,"{'cmn_Hani': 0.59, 'eng_Latn': 0.27}",0.828811,0.00,0.27,0.59,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,me5_cmn_Hani,[cmn_Hani],mhr_Cyrl,Step50+sbeam8,"{'cmn_Hani': 0.61, 'eng_Latn': 0.13}",0.807578,0.00,0.13,0.61,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2502,me5_cmn_Hani,[cmn_Hani],mhr_Cyrl,Base,"{'cmn_Hani': 0.71, 'eng_Latn': 0.08}",0.826749,0.00,0.08,0.71,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2504,me5_cmn_Hani,[cmn_Hani],fin_Latn,Step1,"{'cmn_Hani': 0.49, 'eng_Latn': 0.27}",0.872439,0.00,0.27,0.49,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2505,me5_cmn_Hani,[cmn_Hani],fin_Latn,Step50+sbeam8,"{'cmn_Hani': 0.44, 'eng_Latn': 0.34}",0.858251,0.00,0.34,0.44,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df["pred_langs"].tolist()

["{'deu_Latn': 0.67, 'eng_Latn': 0.06}",
 "{'deu_Latn': 0.69, 'eng_Latn': 0.07}",
 "{'deu_Latn': 0.64}",
 "{'cmn_Hani': 0.66, 'eng_Latn': 0.23}",
 "{'cmn_Hani': 0.59, 'eng_Latn': 0.27}",
 "{'cmn_Hani': 0.64, 'jpn_Jpan': 0.08, 'eng_Latn': 0.2}",
 "{'cmn_Hani': 0.73, 'eng_Latn': 0.2}",
 "{'cmn_Hani': 0.7, 'eng_Latn': 0.22}",
 "{'cmn_Hani': 0.56, 'jpn_Jpan': 0.07, 'eng_Latn': 0.27}",
 "{'cmn_Hani': 0.57, 'eng_Latn': 0.35}",
 "{'cmn_Hani': 0.55, 'eng_Latn': 0.36}",
 "{'jpn_Jpan': 0.09, 'eng_Latn': 0.37, 'cmn_Hani': 0.46}",
 "{'cmn_Hani': 0.52, 'eng_Latn': 0.24}",
 "{'cmn_Hani': 0.48, 'eng_Latn': 0.28}",
 "{'cmn_Hani': 0.21, 'eng_Latn': 0.24, 'jpn_Jpan': 0.36}",
 "{'eng_Latn': 0.45, 'cmn_Hani': 0.27, 'deu_Latn': 0.06}",
 "{'eng_Latn': 0.48, 'cmn_Hani': 0.21, 'deu_Latn': 0.06}",
 "{'cmn_Hani': 0.23, 'jpn_Jpan': 0.06, 'eng_Latn': 0.48}",
 "{'cmn_Hani': 0.47, 'eng_Latn': 0.4}",
 "{'cmn_Hani': 0.42, 'eng_Latn': 0.47}",
 "{'cmn_Hani': 0.28, 'eng_Latn': 0.39, 'jpn_Jpan': 0.22}",
 "{'cmn_Hani': 0.

In [89]:
glotto2code

{'amha1245': 'amh',
 'finn1318': 'fin',
 'hebr1245': 'heb',
 'hind1269': 'hin',
 'hung1274': 'hun',
 'kore1280': 'kor',
 'malt1254': 'mlt',
 'mand1415': 'cmn',
 'nucl1301': 'tur',
 'nucl1643': 'jpn',
 'panj1256': 'pan',
 'stan1318': 'arb',
 'sinh1246': 'sin',
 'guja1252': 'guj',
 'east2328': 'mhr',
 'stan1295': 'deu',
 'urdu1245': 'urd',
 'kaza1248': 'kaz'}

In [8]:
data_file = "../datasets/inversion_language_confusion/line_level/multi/train_data.csv"
df = pd.read_csv(data_file)

In [11]:
df.columns

Index(['model', 'training', 'eval_lang', 'step', 'pred_langs', 'emb_cos_sim',
       'eval_lang_encoded', 'script', 'family', 'word_order', 'script_lr',
       'training_script_lr', 'arb_Arab', 'cmn_Hani', 'deu_Latn', 'guj_Gujr',
       'heb_Hebr', 'hin_Deva', 'jpn_Jpan', 'kaz_Cyrl', 'mon_Cyrl', 'pan_Guru',
       'tur_Latn', 'urd_Arab', 'step_Base', 'step_Labels', 'step_Step1',
       'step_Step50+sbeam8'],
      dtype='object')

In [12]:
df = df.drop(columns=["word_order", "emb_cos_sim"])

In [81]:
df_gb = pd.read_csv("../datasets/inversion_language_confusion/gb_features.csv")

In [83]:
glotto2code = dict(zip(df_gb["Lang_ID"], df_gb["iso639P3code"]))

In [85]:
glotto2code2 = dict(zip(df_wals["Glottocode"], df_wals["iso639P3code"]))

In [86]:
glotto2code.update(glotto2code2)

In [88]:
len(glotto2code)

18

In [20]:
df_gb = df_gb.fillna(-1)

In [16]:
df_wals= pd.read_csv("../datasets/inversion_language_confusion/wals_features.csv")

In [23]:
df_gb.index=df_gb["iso639P3code"]
df_gb.at["amh", "GB020"]

np.int64(1)

In [22]:
gb_features = [x for x in df_gb.columns if x.startswith("GB")]
len(gb_features)

195

In [21]:
df_gb.head(2)

,Lang_ID,GB020,GB021,GB022,GB023,GB024,GB025,GB026,GB027,GB028,...,GB422,GB430,GB431,GB432,GB433,GB519,GB520,GB521,GB522,iso639P3code
0,amha1245,1,1,1,1,1,1,-1.0,-1.0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,amh
1,finn1318,0,0,0,0,1,1,0.0,1.0,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,fin


In [25]:
df_gb = df_gb[gb_features]

In [28]:
lang2feat_vecs = {}
for lang, row in df_gb.iterrows():
    row_features = row.to_numpy()
    lang2feat_vecs[lang]=row_features

In [32]:
def compare_feature_each(i, j):
    if i!=-1 and j!=-1:
        if i==j:
            return 1
        else:
            return 0
    else:
        return -1
        

In [35]:
from itertools import combinations
from tqdm import tqdm

In [34]:
langs = df_gb.index.tolist()
print(langs)

['amh', 'fin', 'heb', 'hin', 'hun', 'kor', 'mlt', 'cmn', 'tur', 'jpn', 'pan', 'arb']


In [37]:
feature_dict = {}
for t in tqdm(combinations(langs, 2)):
    p = sorted(t)
    l1,l2=p
    feature_dict[tuple(p)]= list(map(compare_feature_each, lang2feat_vecs[l1], lang2feat_vecs[l2]))

66it [00:00, 6465.43it/s]


# apply feature dictionary to the data

In [45]:
def compare_elementwise(i,j):
    if i!=-1 and j!=-1:
        if i==1 or j==1:
            return 1
        else:
            return 0
    else:
        return -1

In [46]:
def get_feature_value(x,y):
    # compare features between training language and eval language.
    feature_list = []
    for i in x:
        p = tuple(sorted([i,y]))
        if p in feature_dict:
            feature_list.append(feature_dict[p])
    if len(feature_list)==2:
        return list(map(compare_elementwise, feature_list[0], feature_list[1]))

    elif len(feature_list)==1:
        return feature_list[0]
    else:
        return [-1 for _ in range(len(gb_features))]
        

In [53]:
df["training"]=df["training"].apply(literal_eval)

In [67]:
df["training_iso3"]= df["training"].apply(lambda i: [x.split("_")[0] for x in i])

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_8960/3476844895.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["training_iso3"]= df["training"].apply(lambda i: [x.split("_")[0] for x in i])


In [68]:
df["eval_lang_iso3"] = df["eval_lang"].apply(lambda x: x.split("_")[0])

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_8960/2286187288.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["eval_lang_iso3"] = df["eval_lang"].apply(lambda x: x.split("_")[0])


In [70]:
df[gb_features]= pd.DataFrame(df.apply(lambda row: get_feature_value(row["training_iso3"], row["eval_lang_iso3"]), axis=1).tolist())

In [71]:
df["check_all_minus_one"]= (df[gb_features]==-1).all(axis=1)

In [76]:
df = df[df["check_all_minus_one"]!=True]

In [77]:
len(df)

730

In [78]:
df

,model,training,eval_lang,step,pred_langs,eval_lang_encoded,script,family,script_lr,training_script_lr,...,GB431,GB432,GB433,GB519,GB520,GB521,GB522,check_all_minus_one,training_iso3,eval_lang_iso3
6,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",heb_Hebr,Step1,{'cmn_Hani': 0.93},6,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,False,"[cmn, jpn]",heb
7,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",heb_Hebr,Step50+sbeam8,{'cmn_Hani': 0.94},6,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,False,"[cmn, jpn]",heb
8,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",heb_Hebr,Base,"{'jpn_Jpan': 0.22, 'cmn_Hani': 0.67, 'eng_Latn...",6,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,False,"[cmn, jpn]",heb
9,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",arb_Arab,Step1,"{'cmn_Hani': 0.86, 'eng_Latn': 0.08}",1,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,False,"[cmn, jpn]",arb
10,me5_cmn_jpn,"[cmn_Hani, jpn_Jpan]",arb_Arab,Step50+sbeam8,"{'cmn_Hani': 0.87, 'eng_Latn': 0.08}",1,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,False,"[cmn, jpn]",arb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,me5_cmn_Hani,[cmn_Hani],hun_Latn,Step50+sbeam8,"{'cmn_Hani': 0.56, 'eng_Latn': 0.14}",8,0,0,1,1,...,1,0,0,0,0,1,1,False,[cmn],hun
1782,me5_cmn_Hani,[cmn_Hani],hun_Latn,Base,"{'cmn_Hani': 0.69, 'eng_Latn': 0.09}",8,0,0,1,1,...,1,0,0,0,0,1,1,False,[cmn],hun
1786,me5_cmn_Hani,[cmn_Hani],fin_Latn,Step1,"{'cmn_Hani': 0.82, 'eng_Latn': 0.1}",4,0,0,1,1,...,1,0,0,0,0,1,1,False,[cmn],fin
1787,me5_cmn_Hani,[cmn_Hani],fin_Latn,Step50+sbeam8,"{'cmn_Hani': 0.76, 'eng_Latn': 0.13}",4,0,0,1,1,...,1,0,0,0,0,1,1,False,[cmn],fin


In [17]:
df_wals

,iso639P3code,Family,Genus,Prefixing vs. Suffixing in Inflectional Morphology,The Associative Plural,Intensifiers and Reflexive Pronouns,Position of Case Affixes,Position of Tense-Aspect Affixes,The Morphological Imperative,The Optative,...,Order of Numeral and Noun,Order of Degree Word and Adjective,Order of Adverbial Subordinator and Clause,Relationship between the Order of Object and Verb and the Order of Adposition and Noun Phrase,Relationship between the Order of Object and Verb and the Order of Adjective and Noun,Polar Questions,Tea,Order of Negative Morpheme and Verb,"Position of Negative Word With Respect to Subject, Object, and Verb",Glottocode
0,mlt,16,61,-1,1,0,-1,-1,2,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,malt1254
1,sin,22,50,-1,0,0,0,1,-1,-1,...,2,0,3,0,4,-1,1,-1,-1,sinh1246
2,tur,39,59,1,0,0,0,1,2,0,...,1,0,3,0,4,0,0,5,5,nucl1301
3,fin,47,88,1,3,0,0,1,2,0,...,1,0,2,3,3,0,1,0,7,finn1318
4,guj,22,50,-1,-1,0,-1,1,2,0,...,1,0,2,0,4,1,0,0,6,guja1252
5,hun,47,234,1,2,0,0,1,0,0,...,1,0,2,3,3,0,1,0,7,hung1274
6,pan,22,50,1,0,0,0,1,2,0,...,1,0,2,0,4,0,0,0,6,panj1256
7,mhr,47,278,1,2,-1,0,1,2,0,...,-1,-1,-1,0,4,-1,0,0,-1,east2328
8,jpn,136,306,1,0,0,1,1,1,0,...,1,0,0,0,4,0,0,5,5,nucl1643
9,deu,22,158,1,1,1,0,1,3,0,...,1,0,2,1,0,5,1,10,1,stan1295
